In [1]:
from pyspark.sql import SparkSession

In [7]:
from pyspark.sql.functions import *


In [2]:
spark = SparkSession.builder \
.appName("Wordcount_Demo") \
.master("yarn") \
.getOrCreate()

25/07/03 14:23:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
df = spark.read\
.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("gs://my_spark_data/students.csv")

In [4]:
df.show()

+----------+------------+--------------------+-----+----+
|student_id|        name|               major|score|year|
+----------+------------+--------------------+-----+----+
|  cca8c1d5|SinhVien 208|          Accounting| 2.92|   2|
|  dcf972ad|SinhVien 288|             History| 3.68|   4|
|  a3aaca46|SinhVien 899|Artificial Intell...| 3.49|   3|
|  a142708c|SinhVien 573|           Chemistry| 3.35|   1|
|  8f713b85|SinhVien 597|             Physics| 2.18|   2|
|  aab8ceff|SinhVien 345|         Mathematics| 3.41|   4|
|  59e8e4cd|SinhVien 607|Artificial Intell...| 2.45|   3|
|  4f9ad69f|SinhVien 679|             History| 3.46|   3|
|  721b14a6|SinhVien 682|             Physics| 2.84|   4|
|  611cb4f2|SinhVien 919|          Literature| 2.22|   2|
|  e5021dc1|SinhVien 960|Electrical Engine...| 2.94|   1|
|  16deb048|SinhVien 757|         Mathematics|  3.5|   3|
|  18ce4d69|SinhVien 761|           Economics|  2.5|   4|
|  5cb5dc0f|SinhVien 487|Biomedical Engine...| 3.12|   1|
|  0de24aa6|Si

In [5]:
df.show(5)

+----------+------------+--------------------+-----+----+
|student_id|        name|               major|score|year|
+----------+------------+--------------------+-----+----+
|  cca8c1d5|SinhVien 208|          Accounting| 2.92|   2|
|  dcf972ad|SinhVien 288|             History| 3.68|   4|
|  a3aaca46|SinhVien 899|Artificial Intell...| 3.49|   3|
|  a142708c|SinhVien 573|           Chemistry| 3.35|   1|
|  8f713b85|SinhVien 597|             Physics| 2.18|   2|
+----------+------------+--------------------+-----+----+
only showing top 5 rows



In [8]:
# calculate score average
df.select(avg("score")).show()

+-----------------+
|       avg(score)|
+-----------------+
|2.954900000000001|
+-----------------+



In [10]:
# find student who has highest score
# method 1
df.orderBy(df["score"].desc()).limit(1).show()
# method 2
max_score = df.agg(max("score")).first()[0] # get highest score
df.filter(df["score"] == max_score).show() # get student who has this that score

+----------+------------+-------+-----+----+
|student_id|        name|  major|score|year|
+----------+------------+-------+-----+----+
|  d0314395|SinhVien 937|Physics|  4.0|   3|
+----------+------------+-------+-----+----+



+----------+------------+-------+-----+----+
|student_id|        name|  major|score|year|
+----------+------------+-------+-----+----+
|  d0314395|SinhVien 937|Physics|  4.0|   3|
+----------+------------+-------+-----+----+



In [11]:
# count students by majors
df.groupBy("major").count().show()

+--------------------+-----+
|               major|count|
+--------------------+-----+
|             Finance|    7|
|Business Administ...|    3|
|           Marketing|    5|
|Artificial Intell...|    6|
|    Computer Science|    4|
|          Literature|    8|
|             History|    6|
|         Mathematics|    5|
|Mechanical Engine...|    3|
|           Economics|   10|
|Electrical Engine...|    3|
|        Data Science|    4|
|          Accounting|    7|
|Biomedical Engine...|    4|
|           Sociology|    6|
|             Physics|    5|
|   Civil Engineering|    4|
|           Chemistry|    5|
|          Psychology|    5|
+--------------------+-----+



In [14]:
# filter students who have score higher than 3.5
df.select("student_id").where(col("score") >= 3.5).show()

+----------+
|student_id|
+----------+
|  dcf972ad|
|  16deb048|
|  0de24aa6|
|  8be7d1c4|
|  9a278044|
|  3974942a|
|  141a04e0|
|  f8067c3b|
|  19dda143|
|  7e9e7ea5|
|  b8157ea4|
|  648e1754|
|  d0314395|
|  45ae2376|
|  d592fa19|
|  8e19d707|
|  795b9eab|
|  bb721465|
|  bebcc8a8|
|  a54b3f43|
+----------+
only showing top 20 rows



In [15]:
# calculate average by major
df.groupBy("major").agg(avg("score").alias("avg_score")).show()

+--------------------+------------------+
|               major|         avg_score|
+--------------------+------------------+
|             Finance|2.8714285714285714|
|Business Administ...|              3.23|
|           Marketing|             3.312|
|Artificial Intell...|3.0416666666666665|
|    Computer Science|              3.04|
|          Literature|            2.9075|
|             History| 2.671666666666667|
|         Mathematics|             3.278|
|Mechanical Engine...| 2.663333333333333|
|           Economics|             2.962|
|Electrical Engine...| 2.776666666666666|
|        Data Science|            2.7925|
|          Accounting|3.1185714285714283|
|Biomedical Engine...|              3.19|
|           Sociology|2.8266666666666667|
|             Physics|             2.896|
|   Civil Engineering|             2.845|
|           Chemistry|             3.026|
|          Psychology|2.6799999999999997|
+--------------------+------------------+

